# Yolox for Object Detection Rice Leaf Diseases

## Install Library

In [1]:
# Yolox Dependencies
!git clone https://github.com/roboflow-ai/YOLOX.git
%cd YOLOX
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .  
!pip uninstall -y torch torchvision torchaudio
# May need to change in the future if Colab no longer uses CUDA 11.0
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'YOLOX'...
remote: Enumerating objects: 786, done.
remote: Total 786 (delta 0), reused 0 (delta 0), pack-reused 786
Receiving objects: 100% (786/786), 5.77 MiB | 20.67 MiB/s, done.
Resolving deltas: 100% (418/418), done.
/content/YOLOX
     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 57 kB 2.7 MB/s             
     |████████████████████████████████| 108 kB 11.3 MB/s            
     |████████████████████████████████| 14.5 MB 9.1 kB/s             
     |████████████████████████████████| 4.5 MB 34.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 466 kB 45.1 MB/s            
  Created wheel for onnx-simplifier: filename=onnx_simplifier-0.3.5-py3-none-any.whl size=12878 sha256=e0f5f5c19e89c568f9b46af0f2b3a9e6c3a129f9b3f914fb5d4022cdb3bec9

### Install Graphics

In [ ]:
%cd /content/
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/content
Cloning into 'apex'...
remote: Enumerating objects: 8463, done.
remote: Counting objects: 100% (550/550), done.
remote: Compressing objects: 100% (345/345), done.
remote: Total 8463 (delta 315), reused 360 (delta 193), pack-reused 7913
Receiving objects: 100% (8463/8463), 14.37 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (5716/5716), done.
/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:245: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.3.1 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Processing /content/apex
  Running command python setup.py egg_info


  torch.__version__  = 1.8.0+cu111


  running egg_info
  creating /tmp/pip-pip-egg-info-q0_dvqe_/apex.egg-info
  writing /tmp/pip-pip-egg-info-q0_dvqe_/apex.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-q0_dvqe_/apex.eg

### Library Pycoco

In [ ]:
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

## Prepare Dataset

In [ ]:
#to get your roboflow code below please follow the link output by this cell
!pip -q install roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="voc", notebook="yolox")

In [ ]:
%cd /content/YOLOX/datasets/

In [ ]:
# 416px for nano or tiny model
!curl -L "https://app.roboflow.com/ds/lyclbMopYW?key=rrM8NXbbzj" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# 460px
# !curl -L "https://app.roboflow.com/ds/gnm4mu6NMX?key=WWUHgPh3UK" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

## Format Data

In [ ]:
%cd /content/YOLOX/datasets/
!ln -s train/ ./VOCdevkit

In [ ]:
%cd /content/YOLOX
%mkdir "/content/YOLOX/datasets/VOCdevkit/VOC2007"
!python3 voc_txt.py "/content/YOLOX/datasets/VOCdevkit/"
%mkdir "/content/YOLOX/datasets/VOCdevkit/VOC2012"
!cp -r "/content/YOLOX/datasets/VOCdevkit/VOC2007/." "/content/YOLOX/datasets/VOCdevkit/VOC2012"

## Inisialisasi Class

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/YOLOX/yolox/data/datasets/voc_classes.py

VOC_CLASSES = (
  "brownspot",
  "healthy",
  "hispa",
  "blast",
)

In [ ]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/YOLOX/yolox/data/datasets/coco_classes.py

COCO_CLASSES = (
  "brownspot",
  "healthy",
  "hispa",
  "blast",
)

In [ ]:
# Atur jumlah kelas
NUM_CLASSES = 4
!sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "/content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py"

## Download Pretrained Weight

In [ ]:
%cd /content/
# versi small
# !wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_s.pth
# versi tiny
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_tiny.pth

## Training Model

In [ ]:
%cd /content/YOLOX/
# versi tiny
!python tools/train.py -f exps/example/yolox_voc/yolox_voc_s.py -d 1 -b 16 --fp16 -o -c /content/yolox_tiny.pth
# versi small
# !python tools/train.py -f exps/example/yolox_voc/yolox_voc_s.py -d 1 -b 16 --fp16 -o -c /content/yolox_s.pth
# -d = device yang digunakan, 1 gpu
# -b = batch size
# --fp
# -c = cache pakai pretrained model : /content/yolox_s.pth
# -o = output / defaultnya : folder YOLOX_OUTPUTS

## Evaluasi Model

In [ ]:
MODEL_PATH = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth.tar"
!python3 tools/eval.py -n  yolox-s -c {MODEL_PATH} -b 64 -d 1 --conf 0.001 -f exps/example/yolox_voc/yolox_voc_s.py

## Tes Model

In [ ]:
# TEST_IMAGE_PATH = "/content/valid/BloodImage_00057_jpg.rf.1ee93e9ec4d76cfaddaa7df70456c376.jpg"
# !python tools/demo.py image -f /content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py -c {MODEL_PATH} --path {TEST_IMAGE_PATH} --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

**Visualisasikan**

In [ ]:
# from PIL import Image
# OUTPUT_IMAGE_PATH = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/vis_res/2021_08_01_19_51_59/BloodImage_00057_jpg.rf.1ee93e9ec4d76cfaddaa7df70456c376.jpg" 
# Image.open(OUTPUT_IMAGE_PATH)